# Seq2Seq LLM Reprompt




In [1]:
from datasets import load_dataset
import pandas as pd 

# df.columns
ds = load_dataset("csv",data_files={"train":'gemma10000.csv'})

/home/user/envs/test_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# df=df[['original_text','gemma_7b_rewritten_text_temp0','rewrite_prompt']]
# df['diff_prompt']='Tell me what prompt converted the following text: \n"""' + df['original_text'] + '"""\n to this text \n"""' + df['gemma_7b_rewritten_text_temp0'] + '"""\n'
#original text prefix

orig_prefix = "Original Text:"

rewrite_prefix = "\nRewritten Text:"

import json
import re

def get_prompt(orig_text, transformed_text):
    message = f"{orig_prefix} {orig_text} {rewrite_prefix} {transformed_text}"
    #message = f"{rewrite_prefix} {transformed_text}"
    return message


# interate over all the rows formate the dataset and store it in a jsonl file
def process_jsonl_file(output_file_path):
    with open(output_file_path, "w") as output_jsonl_file:
        for item in ds["train"]:
            json_object = {
                #"text": get_prompt(item["original_text"],item['gemma_7b_rewritten_text_temp0']),
                "text": get_prompt(item["original_text"],item['rewritten_text']),
                "output": item['rewrite_prompt']
            }
            output_jsonl_file.write(json.dumps(json_object) + "\n")

# Provide the path where you want to save the formatted dataset
process_jsonl_file("./training_dataset.jsonl")


In [3]:
from datasets import Dataset
train_dataset = load_dataset('json', data_files='./training_dataset.jsonl', split="train")

train_dataset=train_dataset.train_test_split(test_size=0.1, seed=37)


Generating train split: 9999 examples [00:00, 185000.38 examples/s]


In [4]:
#model_checkpoint = "google/flan-t5-base"
model_checkpoint = "facebook/bart-large"

In [5]:
from datasets import load_dataset
from evaluate import load

metric = load("rouge")

2024-03-24 00:45:45.162052: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 00:45:45.162096: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 00:45:45.163110: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 00:45:45.168110: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-24 00:45:45.802480: W tensorflow/compiler/tf2

In [6]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, device_map="auto")

In [7]:
tokenizer.save_pretrained("./Seq2Seq_bart-large")

('./Seq2Seq_bart-large/tokenizer_config.json',
 './Seq2Seq_bart-large/special_tokens_map.json',
 './Seq2Seq_bart-large/vocab.json',
 './Seq2Seq_bart-large/merges.txt',
 './Seq2Seq_bart-large/added_tokens.json',
 './Seq2Seq_bart-large/tokenizer.json')

In [8]:
from transformers import AutoConfig

config=AutoConfig.from_pretrained(model_checkpoint)
config.save_pretrained('./Seq2Seq_bart-large')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [9]:
config

BartConfig {
  "_name_or_path": "facebook/bart-large",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 1024,
  "mode

In [10]:
max_input_length = 1024
max_target_length = 16

In [11]:
def preprocess_function(examples):
    model_inputs = tokenizer(examples["text"], max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["output"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [12]:
tokenized_dataset = train_dataset.map(preprocess_function, batched=True)

Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 7227.51 examples/s]


In [13]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'output', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 8999
    })
    test: Dataset({
        features: ['text', 'output', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})

In [14]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, device_map="auto")

In [15]:
batch_size = 4
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-reprompt",
    evaluation_strategy = "steps",
    learning_rate=2e-6,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=20,
    predict_with_generate=True,
    fp16=True,
    #fp16_full_eval=True,
    load_best_model_at_end=True
)

In [16]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [17]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [18]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],    
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/home/user/envs/test_env/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [19]:
trainer.train()

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
500,1.959300,1.308410,47.886400,31.268700,47.371000,47.414800,12.919000
1000,1.309000,1.159242,47.703300,30.273600,47.186300,47.220400,14.767000
1500,1.145300,1.064047,49.719000,32.970900,49.205000,49.233500,13.936000
2000,1.059900,0.959474,51.807700,35.497400,51.333800,51.396400,13.224000
2500,0.964900,0.897917,51.506500,35.037400,51.035400,51.076500,13.439000
3000,0.878300,0.851075,54.178900,39.235100,53.804400,53.853600,13.178000
3500,0.851800,0.797945,54.691500,39.584700,54.341400,54.397800,13.573000
4000,0.810100,0.764997,55.722000,40.591700,55.391400,55.453000,13.645000
4500,0.741700,0.755768,55.661700,41.199800,55.426600,55.401400,13.862000
5000,0.711700,0.724504,56.548400,41.503300,56.292000,56.316600,13.132000


/home/user/envs/test_env/lib/python3.10/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
/home/user/envs/test_env/lib/python3.10/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length 

TrainOutput(global_step=45000, training_loss=0.39413419172498915, metrics={'train_runtime': 14431.1857, 'train_samples_per_second': 12.472, 'train_steps_per_second': 3.118, 'total_flos': 1.546447092691845e+17, 'train_loss': 0.39413419172498915, 'epoch': 20.0})

In [20]:
trainer.save_model('Seq2Seq_bart-large')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [21]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained('./Seq2Seq_bart-large', device_map="auto")
#tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large', device_map="auto")
model = AutoModelForSeq2SeqLM.from_pretrained('./Seq2Seq_bart-large', device_map="auto")

In [22]:
def pred(x):
    input_ids = tokenizer(x, return_tensors='pt').input_ids.to('cuda')
    outputs = model.generate(input_ids, max_new_tokens=max_target_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [23]:
print(pred(train_dataset["train"][0]["text"]))
print(train_dataset["train"][0]["output"])
print(train_dataset["train"][0]["text"])

Change the text into a TV shopping network product pitch
Turn the content into a reality TV show pitch
Original Text: Unless otherwise stated, the content of this page is licensed under Creative Commons Attribution-ShareAlike 3.0 License The SCP Foundation Wiki Sigma-9 theme and style was designed by Aelanna , and used under the Creative Commons Attribution-ShareAlike 3.0 license ( CC BY-SA 3.0 ). Branches of the SCP Foundation The results of the Community Survey 2018 are here ! The Review Team needs YOU as a new recruit! The International Translation Archive is now represented on the English Wiki and regularly uploads translations from INT to EN! The International Translation Archive contains translations of SCP articles, tales, canons and other articles 
Rewritten Text: ## SCP Foundation: The Global Hunt Begins!

**Introducing the International Translation Archive, a collaborative platform that bridges language barriers and expands our understanding of the SCP Foundation universe!**


In [29]:
i=72
print(pred(train_dataset["train"][i]["text"]))
print(train_dataset["train"][i]["output"])
print(train_dataset["train"][i]["text"])

Rewrite the text in reverse word order
Rewrite the text in reverse word order
Original Text: Please enable Javascript to watch this video SEATTLE -- A hero in America's war in Afghanistan is now delivering for his 1-year-old nephew. Little Hudson Hill is alive, thanks to his uncle, Lt. Col. Trevor Hill, who donated a part of his liver as a living donor. Trevor underwent surgery at UW Medical Center. Hudson's surgery was 2 miles away at Seattle Children's Hospital. Trevor is an Army Green Beret and says he was motivated to help his nephew, in part, because others saved his life in the battlefied. But he also cites something else -- survivor's guilt. Surgeons removed 
Rewritten Text: 
Sure, here's the reversed text:

Lt. Col. Trevor Hill donated a part of his liver to save his 1-year-old nephew, Hudson Hill. In the battle-stricken war in Afghanistan, he was motivated by the fact that others had saved his life, which was what drove him to help Hudson. Surgeons removed two miles of tissue 